# Attention
First try at processing attention files. Since abandonded.

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
print(datetime.now())

2022-08-10 10:46:35.038735


In [2]:
BASE_PATH='/home/jrm/Adjeroh/Glioma/August_Run/CellProfilerPerPatient/'
ATTENTION_DIR='/home/jrm/Adjeroh/Glioma/August_Run/Attention/'
FILENAMES=[
'Process100_Image.csv',
'Process100_Cells.csv',
'Process100_ExpandCells.csv',
'Process100_Experiment.csv',
'Process100_MergeRBC.csv',
'Process100_Nucleus.csv',
'Process100_RBC.csv',
'Process100_ShrinkRBC.csv',
'Process100_Tissue.csv']
IMAGE_FILE=FILENAMES[0]
TRACKING_FILE='PatchTracking.csv'
# Patch filename format: TCGA-06-0129-01Z-00-DX1_5400_5100.png
# For WSI ID, use first 23 letters.
# For patient or case ID, use first 19 letters. 
LEN_CASE_ID=19
LEN_WSI_ID=23

In [24]:
# Original imagenum: patch number, starting at 1 each on each CellProfiler run e.g. Output5.1
#    Since CP ran 8 different times, 8 patients have patch numbers starting at 1, but most start higher.
#    The original imagenum is present in the attention files.
# New imagenum: patch number, starting at 1 for each patient, regardles of when CellProfiler ran
#    This number is present in the Image.csv and Nucleus.csv files organized per patient.
# Converter: a dict that maps (patient,org_imagenum) to new_imagenum
def load_imagenum_converter():
    df = pd.read_csv(BASE_PATH+TRACKING_FILE)
    converter={}
    for ndx,data in df.iterrows():
        patient=data['patient_directory']
        orig_imagenum=int(data['orig_imagenum'])
        new_imagenum=int(data['new_imagenum'])
        key = (patient,orig_imagenum)
        val = new_imagenum
        converter[key] = val
    return converter
converter = load_imagenum_converter()

In [35]:
def get_attention_df(fold,multiclass,cancer,train):
    # We have filenames like cp_datasets_bcls_fold4_test_class0_bbox_stats.csv
    mb = 'b' # binary
    if multiclass:
        mb = 'm'  
    usage = 'test'   # validation
    if train:
        usage = 'train'
    filename=f"cp_datasets_{mb}cls_fold{fold}_{usage}_class{cancer}_bbox_stats.csv"
    fullpath = ATTENTION_DIR + filename
    # This CSV has leading blanks on every column but the first.
    df = pd.read_csv(fullpath,skipinitialspace=True)
    # Add a patient column for convenience.
    patient_column = df['ImageName'].astype(str).str[:LEN_CASE_ID]
    df['Patient'] = patient_column
    # Tried apply(lambda) to replace ImageNumber but it was very slow.
    # Client code using this df must remember to convert orig to new.
    return df
df = get_attention_df(0,True,0,True)
df

,ImageName,ImageNumber,ObjectNumber,MeanTemp,MedianTemp,MinTemp,MaxTemp,StdTemp,Patient
0,TCGA-02-0004-01Z-00-DX1_4800_3600.png,298,1,0.461643,0.458882,0.399096,0.540614,0.033000,TCGA-02-0004-01Z-00
1,TCGA-02-0004-01Z-00-DX1_4800_3600.png,298,2,0.529523,0.530288,0.448266,0.609818,0.035756,TCGA-02-0004-01Z-00
2,TCGA-02-0004-01Z-00-DX1_4800_3600.png,298,3,0.494011,0.495230,0.396912,0.598675,0.044906,TCGA-02-0004-01Z-00
3,TCGA-02-0004-01Z-00-DX1_4800_3600.png,298,4,0.750842,0.750409,0.710556,0.798023,0.022095,TCGA-02-0004-01Z-00
4,TCGA-02-0004-01Z-00-DX1_4800_3600.png,298,5,0.616764,0.621533,0.474238,0.735776,0.055920,TCGA-02-0004-01Z-00
...,...,...,...,...,...,...,...,...,...
786956,TCGA-28-1749-01Z-00-DX1_10200_9000.png,44261,23,0.870074,0.870077,0.826139,0.913993,0.023133,TCGA-28-1749-01Z-00
786957,TCGA-28-1749-01Z-00-DX1_10200_9000.png,44261,24,0.733821,0.730985,0.718110,0.764831,0.011261,TCGA-28-1749-01Z-00
786958,TCGA-28-1749-01Z-00-DX1_10200_9000.png,44261,25,0.704033,0.701771,0.692021,0.731890,0.009334,TCGA-28-1749-01Z-00
786959,TCGA-28-1749-01Z-00-DX1_10200_9000.png,44261,26,0.716644,0.716222,0.716221,0.719934,0.000981,TCGA-28-1749-01Z-00


In [14]:
def get_profiler_df(fold,multiclass,cancer,train):
    return None

In [15]:
def load_X_y(attention_df,profiler_df):
    X = None
    y = None
    return X,y

In [18]:
def load_one_experiment(fold,multiclass,cancer):
    train = True
    attention = get_attention_df(fold,multiclass,cancer,train)
    profiler = get_profiler_df(fold,multiclass,cancer,train)
    X_train,y_train = load_X_y(attention,profiler)
    # repeat for validation
    X_valid = None
    y_valid = None
    return X_train,y_train,X_valid,y_valid

In [20]:
fold = 0
multiclass = True
cancer = 0
X_train,y_train,X_valid,y_valid = load_one_experiment(fold,multiclass,cancer)
print('Train shape',X_train.shape)

AttributeError: 'NoneType' object has no attribute 'shape'